In [1]:
import random
from collections import defaultdict

# Effect registry placeholder
card_effects = {}

def build_card_db(card_df):
    """
    Convert a DataFrame of card info into a dictionary with card names as keys.
    Each value is a dictionary of the card's attributes.
    """
        # Apply rewards to Future Fusion weights
    if results:
        last_state = setup_simulation(
            card_df=card_df,
            deck_card_names=deck_card_names,
            hand_size=hand_size,
            forced_opening=forced_opening,
            extra_deck=extra_deck
        )
        last_state.initialize_future_fusion_weights()
        for card, value in future_fusion_rewards.items():
            last_state.future_fusion_weights[card] += value / num_runs  # normalize influence

        # Apply rewards to Foolish Burial weights
    if results:
        for card in future_fusion_rewards:
            future_fusion_rewards[card] /= num_runs

        last_state = setup_simulation(
            card_df=card_df,
            deck_card_names=deck_card_names,
            hand_size=hand_size,
            forced_opening=forced_opening,
            extra_deck=extra_deck
        )
        last_state.initialize_future_fusion_weights()
        last_state.initialize_foolish_weights()

        for card, value in future_fusion_rewards.items():
            last_state.future_fusion_weights[card] += value

        foolish_sends = [e["card"] for e in last_state.card_events if e["action"] == "send" and e["source"] == "deck"]
        for card in foolish_sends:
            last_state.foolish_weights[card] += reward / num_runs

row['Name']: row.drop(labels=['Name']).to_dict() for _, row in card_df.iterrows()}

class GameState:
    def initialize_allure_weights(self):
        """Initialize banish preference weights for Allure of Darkness."""
        self.allure_weights = {}
        if not hasattr(self, 'card_db'):
            return
        for card in self.deck:
            if self.card_db.get(card, {}).get("Attribute") == "DARK":
                self.allure_weights[card] = 1

    def evolve_allure_weights(self, learning_rate=0.1, reward_cards=None):
        if not hasattr(self, 'allure_weights'):
            self.initialize_allure_weights()
        reward_cards = reward_cards or []
        for card in reward_cards:
            if card in self.allure_weights:
                self.allure_weights[card] += learning_rate

    def log_allure_weights(self):
        return {k: round(v, 2) for k, v in sorted(self.allure_weights.items(), key=lambda x: -x[1])}
    def initialize_foolish_weights(self):
        """Initialize per-card weights for Foolish Burial targets (default = 1)."""
        self.foolish_weights = {}
        if not hasattr(self, 'card_db'):
            return
        for card in self.deck:
            if self.card_db.get(card, {}).get("Type") == "Monster":
                self.foolish_weights[card] = 1

    def evolve_foolish_weights(self, learning_rate=0.1, reward_cards=None):
        """Simple evolution: increase weights for effective cards used in past runs."""
        if not hasattr(self, 'foolish_weights'):
            self.initialize_foolish_weights()
        reward_cards = reward_cards or []
        for card in reward_cards:
            if card in self.foolish_weights:
                self.foolish_weights[card] += learning_rate

    def log_foolish_weights(self):
        """Utility to inspect learned weights for Foolish Burial."""
            # Log evolved weights after learning
    print("
Future Fusion Weights:")
    print(last_state.log_future_fusion_weights())
    print("
Foolish Burial Weights:")
    print(last_state.log_foolish_weights())

        # Log evolved weights after learning
    future_log = last_state.log_future_fusion_weights()
    foolish_log = last_state.log_foolish_weights()

    # Store for graphing if desired
    if not hasattr(last_state, 'weight_log'):
        last_state.weight_log = []
    last_state.weight_log.append({
        'future_fusion': future_log,
        'foolish_burial': foolish_log
    })

    print("
Future Fusion Weights:")
    print(future_log)
    print("
Foolish Burial Weights:")
    print(foolish_log)

    return {k: round(v, 2) for k, v in sorted(self.foolish_weights.items(), key=lambda x: -x[1])}
    def initialize_future_fusion_weights(self):
        """Initialize per-card weights for Future Fusion targeting (default = 1)."""
        self.future_fusion_weights = {}
        if not hasattr(self, 'card_db'):
            return
        for card in self.deck:
            if self.card_db.get(card, {}).get("Type") == "Dragon":
                self.future_fusion_weights[card] = 1

    def evolve_future_fusion_weights(self, learning_rate=0.1, reward_cards=None):
        """Simple evolution: increase weights for effective cards used in past runs."""
        if not hasattr(self, 'future_fusion_weights'):
            self.initialize_future_fusion_weights()
        reward_cards = reward_cards or []
        for card in reward_cards:
            if card in self.future_fusion_weights:
                self.future_fusion_weights[card] += learning_rate

    def log_future_fusion_weights(self):
        """Utility to inspect learned weights (for analysis/debugging)."""
        return {k: round(v, 2) for k, v in sorted(self.future_fusion_weights.items(), key=lambda x: -x[1])}
    def end_phase(self):
        # Resolve queued end-phase effects (Super Rejuvenation, etc.)
        if hasattr(self, "pending_end_phase_effects"):
            for effect_fn in self.pending_end_phase_effects:
                effect_fn()
        self.resolve_triggers()

        # Enforce hand size limit (max 6)
        if len(self.hand) > 6:
            excess = len(self.hand) - 6
            discarded = self.hand[-excess:]
            for card in discarded:
                self.discard_card(card)
            self.log.append(f"End Phase: Discarded {excess} card(s) to meet hand size limit")

    def draw(self):
        if self.deck:
            card = self.deck.pop(0)
            self.hand.append(card)
            self.log.append(f"Drew {card}")
            return card
        return None

    def play_card(self, card_name):
        if card_name in self.hand:
            self.hand.remove(card_name)
            self.log.append(f"Played {card_name}")
            return card_name
        return None

    def activate_effect(self, card_name):
        if card_name not in self.hand:
            self.log.append(f"Attempted to activate {card_name}, but it's not in hand")
            return

        effect_fn = card_effects.get(card_name)
        if effect_fn:
            self.play_card(card_name)
            effect_fn(self)
            self.resolve_triggers()
        else:
            self.log.append(f"No effect defined for {card_name}")

    def discard_card(self, card_name):
        if card_name in self.hand:
            self.hand.remove(card_name)
            self.graveyard.append(card_name)
            self.card_events.append({"card": card_name, "action": "discard", "source": "hand"})
            self.log.append(f"Discarded {card_name}")
            if card_name == "The White Stone of Legend":
                self.trigger_queue.append(self.trigger_white_stone_of_legend)
        else:
            raise ValueError(f"Cannot discard {card_name} — not in hand")

    def tribute_card(self, card_name):
        if card_name in self.field:
            self.field.remove(card_name)
            self.graveyard.append(card_name)
            self.card_events.append({"card": card_name, "action": "tribute", "source": "field"})
            self.log.append(f"Tributed {card_name}")
            if card_name == "The White Stone of Legend":
                self.trigger_queue.append(self.trigger_white_stone_of_legend)
        else:
            raise ValueError(f"Cannot tribute {card_name} — not on field")

    def send_to_graveyard(self, card_name, source="unknown"):
        self.graveyard.append(card_name)
        self.card_events.append({"card": card_name, "action": "send", "source": source})
        self.log.append(f"Sent {card_name} to graveyard from {source}")
        if card_name == "The White Stone of Legend":
            self.trigger_queue.append(self.trigger_white_stone_of_legend)

    def resolve_triggers(self):
        while self.trigger_queue:
            effect_fn = self.trigger_queue.pop(0)
            effect_fn()

    def trigger_white_stone_of_legend(self):
        blue_eyes_name = "Blue-Eyes White Dragon"
        if blue_eyes_name in self.deck:
            self.deck.remove(blue_eyes_name)
            self.add_to_hand(blue_eyes_name)
            self.log.append("White Stone of Legend triggered: added Blue-Eyes White Dragon to hand")
        else:
            self.log.append("White Stone of Legend triggered but no Blue-Eyes White Dragon in deck")

    def banish(self, card_name):
        self.banished.append(card_name)
        self.log.append(f"Banished {card_name}")

    def add_to_hand(self, card_name):
        self.hand.append(card_name)
        self.log.append(f"Added {card_name} to hand")

    def combo_complete(self, combo_cards):
        return all(card in self.hand for card in combo_cards)

    def choose_mallet_returns(self):
        # Smarter mallet logic based on usefulness relative to hand + deck state
        if not self.card_db:
            return self.hand.copy()

        cards_to_mulligan = []
        remaining_consonance = self.deck.count("Cards of Consonance")

        for card in self.hand:
            info = self.card_db.get(card, {})
            score = 0

            # Card-specific logic
            if card == "The White Stone of Legend" and remaining_consonance > 0:
                score += 10  # Very desirable if combo potential exists

            # General heuristics
            if info.get("Type") == "Dragon":
                score += 2
            if info.get("Tuner") is True:
                score += 1
            if info.get("Level") == 8:
                score += 1
            if card in ["Cards of Consonance", "Trade-In", "Instant Fusion"]:
                score += 3

            # Reject cards with low scores
            if score < 3:
                cards_to_mulligan.append(card)

        return cards_to_mulligan

    def choose_foolish_target(self):
        if not self.card_db:
            return None

        weights = getattr(self, "foolish_weights", {})
        valid_targets = [card for card in self.deck if self.card_db.get(card, {}).get("Type") == "Monster"]
        scored = [(card, weights.get(card, 1)) for card in valid_targets]
        scored.sort(key=lambda x: x[1], reverse=True)

        return scored[0][0] if scored else None

        # Priority 1: White Stone if Blue-Eyes still in deck
        if "The White Stone of Legend" in self.deck and "Blue-Eyes White Dragon" in self.deck:
            return "The White Stone of Legend"

        # Priority 2: Enable Pot of Avarice (need exactly 5 monsters in grave)
        monster_graveyard = [card for card in self.graveyard if self.card_db.get(card, {}).get("Type") == "Monster"]
        if len(monster_graveyard) == 4:
            for card in self.deck:
                if self.card_db.get(card, {}).get("Type") == "Monster":
                    return card

        # Priority 3: Thin low-value cards (not tuner, not dragon, not level 8)
        bad_targets = []
        for card in self.deck:
            info = self.card_db.get(card, {})
            if info.get("Type") == "Monster":
                if not info.get("Tuner") and info.get("Type") != "Dragon" and info.get("Level") not in [8, 1, 4]:
                    bad_targets.append(card)

        if bad_targets:
            return bad_targets[0]  # Thin the first weak draw target

        return None

        # Priority 1: White Stone if Blue-Eyes still in deck
        if "The White Stone of Legend" in self.deck and "Blue-Eyes White Dragon" in self.deck:
            return "The White Stone of Legend"

        # Priority 2: Thin low-value cards (not tuner, not dragon, not level 8)
        bad_targets = []
        for card in self.deck:
            info = self.card_db.get(card, {})
            if info.get("Type") == "Monster":
                if not info.get("Tuner") and info.get("Type") != "Dragon" and info.get("Level") not in [8, 1, 4]:
                    bad_targets.append(card)

        if bad_targets:
            return bad_targets[0]  # Thin the first weak draw target

        return None

    def summary(self):
        combo_core = [
            "Red-Eyes Darkness Metal Dragon",
            "Red-Eyes Darkness Metal Dragon",
            "Debris Dragon",
            "Instant Fusion"
        ]
        has_core = all(card in self.hand for card in combo_core)
        has_fifth = any(card in self.hand for card in ["Heavy Storm", "Giant Trunade"])
        return {
            "deck_remaining": len(self.deck),
            "combo_complete": has_core and has_fifth,
            "hand": self.hand.copy(),
            "log": self.log.copy(),
            "card_events": self.card_events.copy()
        }

# Example effect implementation

def effect_cards_of_consonance(state: GameState):
    if not hasattr(state, 'card_db'):
        state.log.append("Cards of Consonance failed: Card database not loaded")
        return

    valid_tuners = [card for card in state.hand
                    if card in state.card_db
                    and state.card_db[card].get("Type") == "Dragon"
                    and state.card_db[card].get("Tuner") == True]

    if valid_tuners:
        prioritized = sorted(valid_tuners, key=lambda c: state.discard_priority.get(c, 0), reverse=True)
        discard_target = prioritized[0]
        state.discard_card(discard_target)
        state.draw()
        state.draw()
        state.log.append(f"Resolved Cards of Consonance: Discarded {discard_target}, drew 2")
    else:
        state.log.append("Cards of Consonance failed: No valid tuner to discard")

def effect_trade_in(state: GameState):
    level_8_targets = [card for card in state.hand
                        if hasattr(state, 'card_db') and card in state.card_db and state.card_db[card].get("Level") == 8]
    if level_8_targets:
        discard_target = level_8_targets[0]  # simple prioritization for now
        state.discard_card(discard_target)
        state.draw()
        state.draw()
        state.log.append(f"Resolved Trade-In: Discarded {discard_target}, drew 2")
    else:
        state.log.append("Trade-In failed: No valid Level 8 monster to discard")


def setup_simulation(card_df, deck_card_names, hand_size=5, forced_opening=None, extra_deck=None):
    """
    Utility to create a GameState from a dataframe and deck list.
    Automatically builds card database.
    """
    card_db = build_card_db(card_df)
    return GameState(
        deck=deck_card_names,
        opening_hand_size=hand_size,
        forced_opening=forced_opening,
        extra_deck=extra_deck,
        card_db=card_db
    )

def run_multiple_simulations(card_df, deck_card_names, combo_cards, num_runs=100, hand_size=5, forced_opening=None, extra_deck=None):
    future_fusion_rewards = defaultdict(float)
    results = []
    combo_hits = 0
    deck_sizes = []
    playable_count = 0

    for _ in range(num_runs):
        state = setup_simulation(
            card_df=card_df,
            deck_card_names=deck_card_names,
            hand_size=hand_size,
            forced_opening=forced_opening,
            extra_deck=extra_deck
        )
        summary = state.summary()
        results.append(summary)

        # Reward logic for Future Fusion sends
        ff_sends = [e["card"] for e in state.card_events if e["action"] == "send" and e["source"] == "deck" and state.card_db.get(e["card"], {}).get("Type") == "Dragon"]
        reward = 0
        if summary["combo_complete"]:
            reward += 1
        if has_playable_consonance or has_playable_tradein or has_redmd_plus_support:
            reward += 0.5
        reward += (40 - summary["deck_remaining"]) / 40  # scaled deck-thinning bonus

        for card in ff_sends:
            future_fusion_rewards[card] += reward

        if summary["combo_complete"]:
            combo_hits += 1
        deck_sizes.append(summary["deck_remaining"])

        # Updated playability logic
        hand = state.hand
        db = state.card_db

        # Helper filters
        dragon_tuners = [card for card in hand if db.get(card, {}).get("Type") == "Dragon" and db.get(card, {}).get("Tuner") == True]
        level_8s = [card for card in hand if db.get(card, {}).get("Level") == 8]
        low_level_dragons = [card for card in hand if db.get(card, {}).get("Type") == "Dragon" and db.get(card, {}).get("Level", 0) <= 4]

        has_playable_consonance = "Cards of Consonance" in hand and dragon_tuners
        has_playable_tradein = "Trade-In" in hand and level_8s
        has_redmd_plus_support = "Red-Eyes Darkness Metal Dragon" in hand and ("Instant Fusion" in hand or low_level_dragons)

        if has_playable_consonance or has_playable_tradein or has_redmd_plus_support:
            playable_count += 1

            # Apply rewards to Allure of Darkness weights
        allure_banishes = [e["card"] for e in state.card_events if e["action"] == "banish" and state.card_db.get(e["card"], {}).get("Attribute") == "DARK"]
        if hasattr(state, 'allure_weights'):
            for card in allure_banishes:
                state.allure_weights[card] += reward / num_runs

        # Log evolved Allure weights after learning
        allure_log = state.log_allure_weights()

    # Consolidated weight logging
    if not hasattr(state, 'weight_log'):
        state.weight_log = []
    state.weight_log.append({
        'future_fusion': state.log_future_fusion_weights(),
        'foolish_burial': state.log_foolish_weights(),
        'allure_of_darkness': allure_log
    })

    print("
Allure of Darkness Weights:")
    print(allure_log)

        return {
        "success_rate": combo_hits / num_runs,
        "average_deck_remaining": sum(deck_sizes) / num_runs,
        "playable_rate": playable_count / num_runs,
        "raw_results": results
    }

def effect_upstart_goblin(state: GameState):
    state.draw()
    state.log.append("Resolved Upstart Goblin: Drew 1 card")



def effect_card_destruction(state: GameState):
    discard_count = len(state.hand)
    for card in state.hand[:]:
        state.discard_card(card)
    for _ in range(discard_count):
        state.draw()
    state.log.append(f"Resolved Card Destruction: Discarded {discard_count} cards and drew {discard_count}")

def effect_magical_mallet(state: GameState):
    if not state.hand:
        state.log.append("Magical Mallet failed: hand is empty")
        return

    redraw_count = len(state.hand)
    state.log.append(f"Resolved Magical Mallet: Shuffled {redraw_count} cards back and drew {redraw_count}")

    # Choose which cards to mulligan using an overridable method
    mulligan = state.choose_mallet_returns()
    for card in mulligan:
        state.hand.remove(card)
        state.deck.append(card)
    random.shuffle(state.deck)

    for _ in range(redraw_count):
        state.draw()

def effect_super_rejuvenation(state: GameState):
    if not hasattr(state, "pending_end_phase_effects"):
        state.pending_end_phase_effects = []

    def end_phase_draw():
        dragon_discards = sum(
            1 for event in state.card_events
            if event["action"] in ("discard", "tribute")
            and state.card_db.get(event["card"], {}).get("Type") == "Dragon"
        )

        if dragon_discards > 0:
            for _ in range(dragon_discards):
                state.draw()
            state.log.append(f"Super Rejuvenation resolved in end phase: Drew {dragon_discards} card(s) from {dragon_discards} dragon discard(s)/tribute(s)")
        else:
            state.log.append("Super Rejuvenation resolved in end phase: No dragon discards/tributes, drew 0")

    state.pending_end_phase_effects.append(end_phase_draw)
    state.log.append("Activated Super Rejuvenation: Effect will resolve in end phase")

def effect_foolish_burial(state: GameState):
    # Delegate to learnable logic to choose the best send target
    if not hasattr(state, "card_db") or not state.deck:
        state.log.append("Foolish Burial failed: No deck or card database")
        return

    chosen_card = state.choose_foolish_target()
    if chosen_card and chosen_card in state.deck:
        state.deck.remove(chosen_card)
        state.send_to_graveyard(chosen_card, source="deck")
        state.log.append(f"Resolved Foolish Burial: Sent {chosen_card} from deck to graveyard")
    else:
        state.log.append("Foolish Burial resolved: No valid target selected or target not in deck")

# Note: Pot of Avarice is illegal to activate if fewer than 2 cards remain in deck before activation
def effect_pot_of_avarice(state: GameState):
    # Prevent activation if fewer than 2 cards remain in the deck
    if len(state.deck) < 2:
        state.log.append("Pot of Avarice failed: Fewer than 2 cards in deck")
        return
    if not hasattr(state, 'card_db'):
        state.log.append("Pot of Avarice failed: No card database")
        return

    # Find 5 monster cards in graveyard
    monster_graveyard = [card for card in state.graveyard if state.card_db.get(card, {}).get("Type") == "Monster"]

    if len(monster_graveyard) < 5:
        state.log.append("Pot of Avarice failed: Not enough monsters in graveyard")
        return

    # Return 5 monster cards to deck
    selected = monster_graveyard[:5]  # placeholder: first 5 monsters
    for card in selected:
        state.graveyard.remove(card)
        state.deck.append(card)
    random.shuffle(state.deck)

    state.draw()
    state.draw()
    state.log.append(f"Resolved Pot of Avarice: Returned {len(selected)} monsters and drew 2 cards")

def effect_future_fusion(state: GameState):
    # Simulate Future Fusion revealing Five-Headed Dragon: send 5 dragons from deck to graveyard
    if not hasattr(state, 'card_db') or not state.deck:
        state.log.append("Future Fusion failed: No deck or card database")
        return

    # Find all dragons in deck
    dragons_in_deck = [card for card in state.deck if state.card_db.get(card, {}).get("Type") == "Dragon"]

    if len(dragons_in_deck) < 5:
        state.log.append("Future Fusion failed: Not enough Dragons in deck")
        return

    # Use card-specific weights to score each dragon (default = 1)
    weights = getattr(state, "future_fusion_weights", {})
    scored = [(card, weights.get(card, 1)) for card in dragons_in_deck]
    scored.sort(key=lambda x: x[1], reverse=True)

    send_choices = [card for card, _ in scored[:5]]

    for card in send_choices:
        state.deck.remove(card)
        state.send_to_graveyard(card, source="deck")

    state.log.append(f"Resolved Future Fusion (Five-Headed Dragon): Sent {', '.join(send_choices)} from deck to graveyard")} from deck to graveyard")

def effect_allure_of_darkness(state: GameState):
    if not hasattr(state, 'card_db'):
        state.log.append("Allure of Darkness failed: No card database")
        return

    # Draw 2 cards first
    state.draw()
    state.draw()
    state.log.append("Allure of Darkness: Drew 2 cards")

    # Find DARK monsters in hand
    darks = [card for card in state.hand if state.card_db.get(card, {}).get("Attribute") == "DARK"]

    if darks:
        weights = getattr(state, 'allure_weights', {})
        scored = [(card, weights.get(card, 1)) for card in darks]
        scored.sort(key=lambda x: x[1])  # lower weight = more likely to banish
        banish_target = scored[0][0]
        state.hand.remove(banish_target)
        state.banish(banish_target)
        state.log.append(f"Allure of Darkness: Banished {banish_target} to keep hand")
    else:
        # No DARKs — discard entire hand
        discarded = state.hand.copy()
        for card in discarded:
            state.discard_card(card)
        state.log.append(f"Allure of Darkness: No DARK monsters — discarded entire hand of {len(discarded)} cards")

# Register effects
card_effects["Cards of Consonance"] = effect_cards_of_consonance
card_effects["Trade-In"] = effect_trade_in
card_effects["Upstart Goblin"] = effect_upstart_goblin
card_effects["Card Destruction"] = effect_card_destruction
card_effects["Magical Mallet"] = effect_magical_mallet
card_effects["Super Rejuvenation"] = effect_super_rejuvenation
card_effects["Foolish Burial"] = effect_foolish_burial
card_effects["Pot of Avarice"] = effect_pot_of_avarice
card_effects["Future Fusion"] = effect_future_fusion
card_effects["Allure of Darkness"] = effect_allure_of_darkness
